In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')

!nvidia-smi

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Sun Apr 14 19:09:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   57C    P0              22W /  72W |      1MiB / 23034MiB |      0%      Default |
|                            

In [2]:
!pip install datasets==2.11.0 --quiet
!pip install accelerate  --quiet
!pip install evaluate   --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [3]:
# Data Preparation
import pandas as pd
import numpy as np
import datasets
data = pd.read_csv('/content/drive/MyDrive/project/Jfleg4-2-4.csv')
data.head()


,input,target
0,So I think we can not live if old people could...,So I think we would not be alive if our ancest...
1,So I think we can not live if old people could...,So I think we could not live if older people d...
2,So I think we can not live if old people could...,So I think we can not live if old people could...
3,So I think we can not live if old people could...,So I think we can not live if old people can n...
4,For not use car .,Not for use with a car .


In [4]:
# Define the output JSONL file name
filename = '/content/drive/MyDrive/project/output.jsonl'

# Iterate through the rows and write each row as a JSON object to the JSONL file
with open(filename, 'w') as jsonl_file:
    for _, row in data.iterrows():
        json_data = row.to_json(orient='columns')
        jsonl_file.write(json_data + '\n')

In [5]:
from transformers import AutoTokenizer
import datasets

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")

# Define tokenize function
def tokenize_function(examples):
    # Concatenate text pairs if necessary
    if "input" in examples and "target" in examples:  # Assuming the presence of 'input' and 'target' in your dataset
        text = [inp + tgt for inp, tgt in zip(examples["input"], examples["target"])]
    else:
        text = examples["text"]

    # Tokenize batch of text
    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        padding='max_length',  # Will pad to the max length in the batch or max_length if provided
        truncation=True,
        max_length=128,  # You can define max length here
        return_tensors="pt",  # Return PyTorch tensors
    )

    return tokenized_inputs

# Load dataset
finetuning_dataset_loaded = datasets.load_dataset("json", data_files=filename, split="train")

# Apply tokenize function
tokenized_dataset = finetuning_dataset_loaded.map(
    tokenize_function,
    batched=True,
    batch_size=1,  # You can set a larger batch size for efficiency
)

# Add labels
tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e324241736493500/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


Map:   0%|          | 0/67645 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/table.py:1373: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


In [6]:
print(tokenized_dataset["input"][0])

print(tokenized_dataset["target"][0])

print(tokenized_dataset["input_ids"][0])


So I think we can not live if old people could not find siences and tecnologies and they did not developped . 
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
[2598, 309, 1158, 359, 476, 417, 3153, 604, 1711, 952, 812, 417, 1089, 256, 7545, 285, 716, 14340, 5970, 285, 597, 858, 417, 372, 652, 80, 1882, 964, 1893, 309, 1158, 359, 651, 417, 320, 9338, 604, 776, 25938, 858, 417, 1287, 23496, 285, 10296, 964, 209, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
# Train test split
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, shuffle=True, seed=123)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 60880
    })
    test: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6765
    })
})


In [8]:
# Before Training
import datasets
import logging
import random
import logging
import torch
import transformers
import pandas as pd

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments, Trainer
model_name = "EleutherAI/pythia-70m"
base_model = AutoModelForCausalLM.from_pretrained(model_name)

device_count = torch.cuda.device_count()
if device_count > 0:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

base_model.to(device)
print(device)

cuda


In [34]:
test_text = test_dataset[1]['input']
max_input_tokens = 1000
max_output_tokens=100
# Tokenize
input_ids = tokenizer.encode(
      test_text,
      return_tensors="pt",
      truncation=True,
      max_length=max_input_tokens
)

# Generate
device = base_model.device
generated_tokens_with_prompt = base_model.generate(input_ids=input_ids.to(device), max_length=max_output_tokens)

# Decode
generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

# Strip the prompt
generated_text_answer = generated_text_with_prompt[0][len(test_text):]


print("input:", test_text)
print(f"Correct answer from dataset: {test_dataset[1]['target']}")
print("Model's answer: ")
print(generated_text_answer)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


input: Out of the six Generation-IV ( Gen-IV ) nuclear reactors that are introduced , the supercritical water-cooled reactor ( SCWR ) appears to show the most potential to be the nuclear power plant for the next generation to come .
Correct answer from dataset: Out of the six Generation-IV ( Gen-IV ) nuclear reactors that are introduced , the supercritical water-cooled reactor ( SCWR ) appears to show the most potential to be the nuclear power plant for the next generation .
Model's answer: 
The reactor is a nuclear power plant that is built by the supercritical water-cooled reactor ( SCWR ) and is a reactor that is built by the supercritical water-cooled reactor ( SCWR ) as well as the supercritical


In [10]:
# finetuning
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
# number of epoch
max_steps = -1

# Save model to this direction
trained_model_name = f"pythia_ft_{max_steps}_steps"
output_dir = '/content/drive/MyDrive/project/' + trained_model_name
save_dir = f'{output_dir}/final'

training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=3,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=64,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  save_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=None)

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)


training_output = trainer.train()

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss
120,1.210200,1.269929
240,1.173900,1.242517
360,1.190300,1.231068
480,1.171300,1.223424
600,1.286400,1.220105


Saved model to: /content/drive/MyDrive/project/pythia_ft_-1_steps/final


In [11]:
device_count = torch.cuda.device_count()
if device_count > 0:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)
finetuned_slightly_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [12]:
# def generate_output(test_question, model, max_input_length, max_new_tokens):
#     # Tokenize
#     input_ids = tokenizer.encode(
#         test_question,
#         return_tensors="pt",
#         truncation=True,
#         max_length=max_input_length  # Make sure to set a proper max_length
#     )

#     # Generate
#     device = model.device
#     generated_tokens_with_prompt = model.generate(
#         input_ids=input_ids.to(device),
#         max_new_tokens=max_new_tokens  # Set max_new_tokens to control the number of generated tokens
#     )

#     # Decode
#     generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

#     # Strip the prompt (if necessary)
#     if test_question in generated_text_with_prompt[0]:
#         generated_text_answer = generated_text_with_prompt[0].replace(test_question, '')
#     else:
#         generated_text_answer = generated_text_with_prompt[0]
#     return generated_text_answer.strip()


In [27]:
model_name = "EleutherAI/pythia-70m"
base_model = AutoModelForCausalLM.from_pretrained(model_name)
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)
finetuned_slightly_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [35]:
max_input_tokens = 1000
max_output_tokens=100
tuned_predicted_text_list = []
actual_test_list = []
base_predicted_text_list = []
for i in range(50):
  test_text = test_dataset[i]['input']
  # Tokenize
  input_ids = tokenizer.encode(
        test_text,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_tokens
  )

  # Generate
  device = base_model.device
  generated_tokens_with_prompt = base_model.generate(input_ids=input_ids.to(device), max_length=max_output_tokens)
  generated_tokens_with_prompt_ft = finetuned_slightly_model.generate(input_ids=input_ids.to(device), max_length=max_output_tokens)
  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)
  generated_text_with_prompt_ft = tokenizer.batch_decode(generated_tokens_with_prompt_ft, skip_special_tokens=True)
  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(test_text):]
  generated_text_answer_ft = generated_text_with_prompt_ft[0][len(test_text):]

  actual_test_list.append(test_dataset[i]['target'])
  tuned_predicted_text_list.append(generated_text_answer_ft)
  base_predicted_text_list.append(generated_text_answer)
  # print("input:", test_text)
  # print(f"Correct answer from dataset: {test_dataset[i]['target']}")
  # print("Base Model's answer: ")
  # print(generated_text_answer)
  # print("Fine-tuned Model's answer: ")
  # print(generated_text_answer_ft)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [40]:
!pip install rouge_score --quiet

In [41]:
import evaluate
bleu = evaluate.load("bleu")

results = bleu.compute(predictions=base_predicted_text_list, references=actual_test_list)
print("Base Model Predictions Results")
print(results)

results = bleu.compute(predictions=tuned_predicted_text_list, references=actual_test_list)
print("Fine Tuned Model Predictions Results")
print(results)
# Load rouge for evaluation
rouge = evaluate.load('rouge')

# Compute ROUGE scores for the base model
base_rouge_results = rouge.compute(predictions=base_predicted_text_list, references=actual_test_list)
print("Base Model Predictions ROUGE Results")
print(base_rouge_results)

# Compute ROUGE scores for the fine-tuned model
tuned_rouge_results = rouge.compute(predictions=tuned_predicted_text_list, references=actual_test_list)
print("Fine Tuned Model Predictions ROUGE Results")
print(tuned_rouge_results)

Base Model Predictions Results
{'bleu': 0.05193351178764942, 'precisions': [0.11827956989247312, 0.052997681351440874, 0.03805995284607612, 0.030489893799246316], 'brevity_penalty': 1.0, 'length_ratio': 2.59425190194421, 'translation_length': 3069, 'reference_length': 1183}
Fine Tuned Model Predictions Results
{'bleu': 0.45538034593301796, 'precisions': [0.5850980392156863, 0.48573757131214346, 0.4161016949152542, 0.36363636363636365], 'brevity_penalty': 1.0, 'length_ratio': 1.0777683854606932, 'translation_length': 1275, 'reference_length': 1183}
Base Model Predictions ROUGE Results
{'rouge1': 0.1599430506827236, 'rouge2': 0.07976037706302078, 'rougeL': 0.14941653978116898, 'rougeLsum': 0.14360796673046228}
Fine Tuned Model Predictions ROUGE Results
{'rouge1': 0.6550223031443061, 'rouge2': 0.5546735492155686, 'rougeL': 0.6483800101418142, 'rougeLsum': 0.6469474629394943}
